In [ ]:
import numpy as np
import torch
import torch.nn as nn
from torchvision import models
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
import pandas as pd

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

Using device: cuda


In [ ]:
MODEL_NAME = "densenet"   # "Model Type"
CHECKPOINT = "densenet121_cifar10_clean.pth"

if MODEL_NAME == "resnet":
    model = models.resnet18(pretrained=False)
    model.fc = nn.Linear(model.fc.in_features, 10)

elif MODEL_NAME == "densenet":
    model = models.densenet121(pretrained=False)
    model.classifier = nn.Linear(model.classifier.in_features, 10)

model.load_state_dict(torch.load(CHECKPOINT, map_location=device))
model.to(device)
model.eval()

DenseNet(
  (features): Sequential(
    (conv0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (norm0): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu0): ReLU(inplace=True)
    (pool0): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (denseblock1): _DenseBlock(
      (denselayer1): _DenseLayer(
        (norm1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu1): ReLU(inplace=True)
        (conv1): Conv2d(64, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (norm2): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu2): ReLU(inplace=True)
        (conv2): Conv2d(128, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      )
      (denselayer2): _DenseLayer(
        (norm1): BatchNorm2d(96, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu

In [ ]:
class CIFAR10C(Dataset):
    def __init__(self, data, labels):
        self.data = data
        self.labels = labels
        self.transform = transforms.Compose([
            transforms.ToTensor(),
            transforms.Normalize((0.5,0.5,0.5),(0.5,0.5,0.5))
        ])

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        img = self.data[idx]
        img = self.transform(img)
        label = self.labels[idx]
        return img, label


In [ ]:
@torch.no_grad()
def evaluate_cifar10c(corruption, severity):
    data = np.load(f"{corruption}.npy")
    labels = np.load("labels.npy")

    start = (severity - 1) * 10000
    end = severity * 10000

    data = data[start:end]
    labels = labels[start:end]

    dataset = CIFAR10C(data, labels)
    loader = DataLoader(
        dataset,
        batch_size=128,
        shuffle=False,
        num_workers=2,
        pin_memory=True
    )

    correct = 0
    total = 0

    for images, targets in loader:
        images = images.to(device)
        targets = targets.to(device)

        outputs = model(images)
        preds = outputs.argmax(1)

        correct += preds.eq(targets).sum().item()
        total += targets.size(0)

    return 100 * correct / total

In [ ]:
clean_acc = 84.41
corruptions = [
    "contrast",
    "fog",
    "motion_blur",
    "gaussian_noise",
    "brightness",
    "jpeg_compression"
]

severities = [3, 5]

results = []

for corr in corruptions:
    for sev in severities:
        acc = evaluate_cifar10c(corr, sev)
        drop = clean_acc - acc

        results.append([
            MODEL_NAME,
            corr,
            sev,
            round(acc, 2),
            round(drop, 2)
        ])

        print(f"{corr.upper()} | Severity {sev}")
        print(f"Accuracy: {acc:.2f}%")
        print(f"Performance Drop: {drop:.2f}%\n")

CONTRAST | Severity 3
Accuracy: 59.21%
Performance Drop: 25.20%

CONTRAST | Severity 5
Accuracy: 30.00%
Performance Drop: 54.41%

FOG | Severity 3
Accuracy: 65.53%
Performance Drop: 18.88%

FOG | Severity 5
Accuracy: 50.34%
Performance Drop: 34.07%

MOTION_BLUR | Severity 3
Accuracy: 57.05%
Performance Drop: 27.36%

MOTION_BLUR | Severity 5
Accuracy: 53.26%
Performance Drop: 31.15%

GAUSSIAN_NOISE | Severity 3
Accuracy: 65.37%
Performance Drop: 19.04%

GAUSSIAN_NOISE | Severity 5
Accuracy: 58.37%
Performance Drop: 26.04%

BRIGHTNESS | Severity 3
Accuracy: 73.27%
Performance Drop: 11.14%

BRIGHTNESS | Severity 5
Accuracy: 69.08%
Performance Drop: 15.33%

JPEG_COMPRESSION | Severity 3
Accuracy: 71.64%
Performance Drop: 12.77%

JPEG_COMPRESSION | Severity 5
Accuracy: 69.91%
Performance Drop: 14.50%



In [ ]:
df = pd.DataFrame(
    results,
    columns=["Model", "Corruption", "Severity", "Accuracy (%)", "Drop (%)"]
)

df = df.sort_values(by="Drop (%)", ascending=False)
print("\nFINAL ROBUSTNESS TABLE")
print("="*40)
print(df)

print("\nMean Performance Drop:",
      round(df["Drop (%)"].mean(), 2), "%")


FINAL ROBUSTNESS TABLE
       Model        Corruption  Severity  Accuracy (%)  Drop (%)
1   densenet          contrast         5         30.00     54.41
3   densenet               fog         5         50.34     34.07
5   densenet       motion_blur         5         53.26     31.15
4   densenet       motion_blur         3         57.05     27.36
7   densenet    gaussian_noise         5         58.37     26.04
0   densenet          contrast         3         59.21     25.20
6   densenet    gaussian_noise         3         65.37     19.04
2   densenet               fog         3         65.53     18.88
9   densenet        brightness         5         69.08     15.33
11  densenet  jpeg_compression         5         69.91     14.50
10  densenet  jpeg_compression         3         71.64     12.77
8   densenet        brightness         3         73.27     11.14

Mean Performance Drop: 24.16 %
